In [1]:
import os
import pandas as pd

sys.path.insert(0, '..\..\Resources\Python-Functions')
import BigQuery

# import transit route outputs
bqclient = BigQuery.getBigQueryClient_TDMScenarios()

In [2]:
# read in manully created file
dfRouteSummaryNames_Revised = pd.read_csv(os.path.join((os.path.join(os.getcwd(),'manual')),'routesummarynames_revised.csv'))
dfRouteSummaryNames_Revised

,NAME_New,ROUTEID,LONGNAME,NAME,numRecords,minYear,maxYear,minMode
0,701_Blue,701,701 - TRAX Blue Line,Blue,62,2019,2050,7
1,703_Red,703,703 - TRAX Red Line,Red,64,2019,2050,7
2,704_Green,704,704 - TRAX Green Line,Green,64,2019,2050,7
3,720_SLine,720,720 - S-Line,Sline,62,2019,2050,7
4,750_FR,750,750 - FrontRunner,RCRT_WEPN,64,2019,2050,8
...,...,...,...,...,...,...,...,...
209,WFRC_082,NaN,WFRC_082 - StGrid_2,StGrid_2,30,2030,2050,5
210,WFRC_083,NaN,WFRC_083 - StGrid_2_BRT,StGrid_2_BRT,24,2032,2050,9
211,WFRC_084,NaN,WFRC_084 - StGrid_2_Core,StGrid_2_Core,4,2032,2032,5
212,WFRC_085,NaN,WFRC_085 - TooeleCor,TooeleCor,1,2050,2050,6


In [ ]:
data = [['BY_2019' ,'v832_2019_ODStationDetail.csv'],
        ['TIP_2024','v832_2024_ODStationDetail.csv'],
        ['RTP_2030','v832_2030_ODStationDetail.csv'],
        ['RTP_2040','v832_2040_ODStationDetail.csv'],
        ['RTP_2050','v832_2050_ODStationDetail.csv']]

dfTotVol = pd.DataFrame()

for df in data:
    dfTemp = pd.read_csv(os.path.join(data_folder, df[1]))
    dfTemp['MODEL'] = df[0]
    display(dfTemp)

    dfLinkLookup = dfTemp[['MODE','NAMEID','NAME','LINKSEQ1','A']].copy()
    dfLinkLookup['LINKSEQ_OF_B'] = dfLinkLookup['LINKSEQ1'] + 1

    dfLinkLookupMax = dfLinkLookup.groupby(['MODE','NAMEID','NAME'],as_index=False).agg(MAXLINKSEQ1=('LINKSEQ1','max'))
    dfLinkLookupMax['MAXLINKSEQp1'] = dfLinkLookupMax['MAXLINKSEQ1'] + 1
    dfLinkLookupWithMaxPlus1 = pd.DataFrame.merge(dfLinkLookup, dfLinkLookupMax, on=('MODE','NAMEID','NAME'), how='left')
    dfLinkLookupWithMaxPlus1.loc[(dfLinkLookupWithMaxPlus1['LINKSEQ1']==0), 'LINKSEQ1'] = dfLinkLookupWithMaxPlus1['MAXLINKSEQp1']
    dfLinkLookupWithMaxPlus1.loc[(dfLinkLookupWithMaxPlus1['LINKSEQ_OF_B']==1), 'LINKSEQ_OF_B'] = 0

    dfLinkLookupWithMaxPlus1 = dfLinkLookupWithMaxPlus1.drop(columns=['MAXLINKSEQ1','MAXLINKSEQp1'])

    dfJoinToGetB = pd.DataFrame.merge(dfLinkLookupWithMaxPlus1, dfLinkLookupWithMaxPlus1, left_on=('MODE','NAMEID','NAME','LINKSEQ_OF_B'), right_on=('MODE','NAMEID','NAME','LINKSEQ1'))
    dfJoinToGetB = dfJoinToGetB[['MODE','NAMEID','NAME','LINKSEQ1_x','A_x','A_y']]
    dfJoinToGetB.columns = ('MODE','NAMEID','NAME','LINKSEQ1','A','B')

    dfTempPrepped = pd.DataFrame.merge(dfJoinToGetB,dfTemp[['MODE','NAMEID','NAME','LINKSEQ1','A','DIST','D_TOT_VOL','MODEL']], on=('MODE','NAMEID','NAME','LINKSEQ1','A'))

    #display(dfLinkLookup)
    #display(dfLinkLookupMax)
    #display(dfLinkLookupWithMaxPlus1)
    #display(dfJoinToGetB)
    display(dfTempPrepped)

    dfTotVol = dfTotVol.append(dfTempPrepped)

    #display(dfLinkLookup.sort_values(by='LINK', ascending=False).drop_duplicates(subset='NAME'))

display(dfTotVol)
display(dfTotVol['D_TOT_VOL'].sum())